# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
ft_dir = os.getenv("PRICE_DATA")
ft_glob = glob(ft_dir+'*/*/*.parquet')
df = dd.read_parquet(ft_glob).compute().reset_index()
df


,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
0,A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613007,4674353,Health Care,Life Sciences Tools & Services,2000
1,A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281460,4765083,Health Care,Life Sciences Tools & Services,2000
2,A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782795,5758642,Health Care,Life Sciences Tools & Services,2000
3,A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344170,2534434,Health Care,Life Sciences Tools & Services,2000
4,A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372868,2819626,Health Care,Life Sciences Tools & Services,2000
...,...,...,...,...,...,...,...,...,...,...,...
2745830,ZTS,2024-06-21,169.380005,171.789993,169.350006,171.089996,171.089996,4899700,Health Care,Pharmaceuticals,2024
2745831,ZTS,2024-06-24,171.710007,174.289993,170.300003,171.839996,171.839996,1885900,Health Care,Pharmaceuticals,2024
2745832,ZTS,2024-06-25,171.789993,172.419998,167.720001,167.979996,167.979996,2293400,Health Care,Pharmaceuticals,2024
2745833,ZTS,2024-06-26,167.210007,171.509995,166.449997,170.899994,170.899994,2053800,Health Care,Pharmaceuticals,2024


In [7]:
type(df)

pandas.core.frame.DataFrame

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [8]:
# Write your code below.
dask_df = dd.from_pandas(df, npartitions=4)

dd_feat = dask_df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1=x['Close'].shift(1),
        Adj_Close_lag=x['Adj Close'].shift(1)
    ).assign(
        returns=lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1,
        hi_lo_range=lambda x: x['High'] - x['Low']
    )
)

C:\Users\James\AppData\Local\Temp\ipykernel_8772\3901578981.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dask_df.groupby('ticker', group_keys=False).apply(


In [9]:
type(dd_feat)

dask_expr._collection.DataFrame

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [12]:
# Write your code below.
import pandas as pd

dd_feat_pd = dd_feat.compute()
dd_feat_pd['rolling_avg_return'] = dd_feat_pd.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)
print(dd_feat_pd.head())


       ticker       Date        Open        High         Low       Close  \
686459    CVX 2018-05-22  129.089996  130.389999  128.100006  128.229996   
686460    CVX 2018-05-23  127.089996  128.789993  126.550003  128.699997   
686461    CVX 2018-05-24  127.400002  127.400002  125.849998  126.610001   
686462    CVX 2018-05-25  124.540001  124.760002  121.220001  122.190002   
686463    CVX 2018-05-29  121.099998  122.430000  120.059998  121.389999   

        Adj Close   Volume  sector             subsector  year  Close_lag_1  \
686459  98.859962  5444200  Energy  Integrated Oil & Gas  2018          NaN   
686460  99.222305  6017200  Energy  Integrated Oil & Gas  2018   128.229996   
686461  97.611000  5906200  Energy  Integrated Oil & Gas  2018   128.699997   
686462  94.203377  8936600  Energy  Integrated Oil & Gas  2018   126.610001   
686463  93.586609  7385700  Energy  Integrated Oil & Gas  2018   122.190002   

        Adj_Close_lag   returns  hi_lo_range  rolling_avg_return  
6

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?<br>

    *No, it is not necessary in my opinion, I guess I don't understand it enough just yet, I even think using dask was not necesary in the 2nd step*<br>

+ Would it have been better to do it in Dask? Why?<br>

    *Based on lecture content, the answer is yes, dask has the capability to handle large datasets efficiently due to its binary nature.*

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.